In [ ]:
!pwd

In [ ]:
from PIL import Image
import numpy as np
from PIL import Image
import os

# Check the current working directory
print("Current Working Directory:", os.getcwd())

# Correct path considering the script location
base_dir = "/Users/amierzuhri/code/JplepeckiBR/PCA/raw_data/facial_segmentation"
image_path = os.path.join(base_dir, "image/0028_0009.jpg")
mask_path = os.path.join(base_dir, "mask/0028_0009.png")

# Try opening the image
try:
    background = Image.open(image_path)
    foreground = Image.open(mask_path)
    print("Images opened successfully.")
except FileNotFoundError as e:
    print(f"Error: {e}")

In [ ]:
background = Image.open(image_path)

In [ ]:
foreground = Image.open(mask_path)

In [ ]:
R = np.array(background)[:,:,0]

In [ ]:
G = np.array(background)[:,:,1]

In [ ]:
B = np.array(background)[:,:,2]

In [ ]:
RGB = np.dstack(((R*np.array(foreground)),(G*np.array(foreground)) ,(B*np.array(foreground))))
RGB[0]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(np.array(background))

In [ ]:
set(np.array(foreground).flatten())

In [ ]:
everything_but_the_face = np.dstack((R*(np.array(foreground)==0),G*(np.array(foreground)==0) ,B*(np.array(foreground)==0)))
plt.imshow(everything_but_the_face )

In [ ]:
skin = np.dstack((R*(np.array(foreground)==1),G*(np.array(foreground)==1) ,B*(np.array(foreground)==1)))
plt.imshow(skin)

In [ ]:
right_brow = np.dstack((R*(np.array(foreground) ==2),G*(np.array(foreground) ==2) ,B*(np.array(foreground)==2)))

In [ ]:
left_brow = np.dstack((R*(np.array(foreground)==3),G*(np.array(foreground)==3) ,B*(np.array(foreground)==3)))

In [ ]:
brows = right_brow + left_brow
plt.imshow(brows)

In [ ]:
right_eye = np.dstack((R*(np.array(foreground)==4),G*(np.array(foreground)==4) ,B*(np.array(foreground)==4)))


In [ ]:
left_eye = np.dstack((R*(np.array(foreground)==5),G*(np.array(foreground)==5) ,B*(np.array(foreground)==5)))

In [ ]:
eyes = left_eye + right_eye
plt.imshow(eyes)

In [ ]:
lowerlip = np.dstack((R*(np.array(foreground)==7),G*(np.array(foreground)==7) ,B*(np.array(foreground)==7)))


In [ ]:
upperlip = np.dstack((R*(np.array(foreground)==9),G*(np.array(foreground)==9) ,B*(np.array(foreground)==9)))


In [ ]:
lips = upperlip + lowerlip
plt.imshow(lips)

In [ ]:
hair = np.dstack((R*(np.array(foreground)==10),G*(np.array(foreground)==10) ,B*(np.array(foreground)==10)))
plt.imshow(hair)

In [ ]:
plt.imshow(np.array(foreground)==10)

In [ ]:
type(hair)

In [ ]:
#face_features = [skin, hair, brows, lips ]


In [ ]:
## Flatten the segment to (num_pixels, 3) where 3 represents the RGB channels
#flat_segment = hair.reshape(-1,3)


In [ ]:
## Filter out zero pixels (background) by checking where any of the channels is non-zero
#non_zero_pixels = flat_segment[np.any(flat_segment != 0, axis=1)]

In [ ]:
## Calculate the mean for each channel
#mean_red = np.mean(non_zero_pixels[:, 0])
#mean_green = np.mean(non_zero_pixels[:, 1])
#mean_blue = np.mean(non_zero_pixels[:, 2])
def average_color(segment):
    """
    Finds the average color of a segment by filtering out background pixels and calculating the mean RGB value.

    Args:
    segment (numpy.ndarray): A 3D numpy array representing a segment of an image (height, width, RGB).

    Returns:
    tuple: A tuple of three integers representing the average RGB color.
    """
    # Flatten the segment to (num_pixels, 3)
    flat_segment = segment.reshape(-1, 3)

    # Filter out zero pixels (background) by checking where any of the channels is non-zero
    non_zero_pixels = flat_segment[np.any(flat_segment != 0, axis=1)]

    # If there are no non-zero pixels, return a default color (e.g., black)
    if len(non_zero_pixels) == 0:
        return (0, 0, 0)

    # Calculate the mean for each channel
    mean_red = np.mean(non_zero_pixels[:, 0])
    mean_green = np.mean(non_zero_pixels[:, 1])
    mean_blue = np.mean(non_zero_pixels[:, 2])

    # Return the mean values as a tuple of integers
    return (int(mean_red), int(mean_green), int(mean_blue))


In [ ]:
average_skin_color = average_color(skin)
average_brows_color = average_color(brows)
average_eyes_color = average_color(eyes)
average_lips_color = average_color(lips)
average_hair_color = average_color(hair)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def euclidean_distance(color1, color2):
    """
    Calculate the Euclidean distance between two RGB colors.
    """
    R1, G1, B1 = color1
    R2, G2, B2 = color2
    return np.sqrt((R1 - R2) ** 2 + (G1 - G2) ** 2)

def knn_closest_colors(target_color, color_palette, k=3):
    """
    Find the K closest colors from a palette to the target color using Euclidean distance.

    Args:
    target_color (tuple): RGB values of the target color.
    color_palette (list): List of RGB tuples representing the color palette.
    k (int): Number of closest colors to find.

    Returns:
    list: The K closest RGB colors from the palette.
    """
    distances = [(color, euclidean_distance(target_color, color)) for color in color_palette]
    distances.sort(key=lambda x: x[1])  # Sort by distance
    return [color for color, _ in distances[:k]]

def visualize_color(color, title):
    """
    Visualize a solid color image.

    Args:
    color (tuple): RGB tuple representing the color to visualize.
    title (str): The title of the plot.
    """
    # Create a 100x100 image of the specified color
    image = np.ones((100, 100, 3))
    image[:, :, 0] *= color[0] / 255.0
    image[:, :, 1] *= color[1] / 255.0
    image[:, :, 2] *= color[2] / 255.0

    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    plt.show()

def visualize_palette(palette, title):
    """
    Visualize a palette of colors in a horizontal bar.

    Args:
    palette (list): List of RGB tuples representing the color palette.
    title (str): The title of the plot.
    """
    # Create an image with a row of color blocks
    palette_image = np.ones((50, len(palette) * 50, 3))
    for i, color in enumerate(palette):
        palette_image[:, i * 50:(i + 1) * 50] = [c / 255.0 for c in color]

    plt.imshow(palette_image)
    plt.title(title)
    plt.axis('off')
    plt.show()

# def analyze_combined_palette_knn(feature_colors, k=3):
#     """
#     Analyzes and finds the K closest colors from predefined palettes to the combined facial feature colors using KNN.

#     Args:
#     feature_colors (list): List of RGB values of the facial feature colors.
#     k (int): Number of nearest neighbors to consider.

#     Returns:
#     None: Prints the closest colors from each palette and the closest overall palette.
#     """
#     spring_palette = [
#         (255, 223, 186), (255, 236, 179), (240, 191, 139), (233, 141, 154),
#         (255, 239, 213), (250, 240, 190), (255, 222, 173), (152, 251, 152),
#         (255, 182, 193), (250, 218, 221), (245, 245, 220), (255, 192, 203),
#         (250, 250, 210), (230, 230, 250), (240, 230, 140)
    # ]

    # summer_palette = [
    #     (176, 224, 230), (250, 240, 230), (224, 255, 255), (255, 228, 225),
    #     (240, 255, 255), (245, 245, 245), (211, 211, 211), (135, 206, 250),
    #     (255, 240, 245), (176, 196, 222), (255, 250, 250), (240, 248, 255),
    #     (123, 104, 238), (143, 188, 143), (255, 245, 238)
    # ]

    # autumn_palette = [
    #     (139, 69, 19), (205, 133, 63), (244, 164, 96), (255, 165, 0),
    #     (210, 105, 30), (165, 42, 42), (222, 184, 135), (184, 134, 11),
    #     (218, 165, 32), (153, 101, 21), (188, 143, 143), (255, 140, 0),
    #     (139, 69, 19), (205, 92, 92), (233, 150, 122)
    # ]

    # winter_palette = [
    #     (0, 0, 139), (255, 255, 255), (128, 0, 0), (47, 79, 79),
    #     (70, 130, 180), (25, 25, 112), (0, 128, 128), (72, 61, 139),
    #     (0, 255, 255), (176, 196, 222), (0, 0, 205), (105, 105, 105),
    #     (0, 100, 0), (112, 128, 144), (0, 191, 255)
    # ]

    # # Calculate the average color across all face features
    # combined_color = tuple(np.mean(feature_colors, axis=0).astype(int))

    # # Get the K closest colors from each palette
    # closest_spring_colors = knn_closest_colors(combined_color, spring_palette, k)
    # closest_summer_colors = knn_closest_colors(combined_color, summer_palette, k)
    # closest_autumn_colors = knn_closest_colors(combined_color, autumn_palette, k)
    # closest_winter_colors = knn_closest_colors(combined_color, winter_palette, k)

    # # Calculate average distance for the K closest colors
    # def average_distance(colors):
    #     return np.mean([euclidean_distance(combined_color, color) for color in colors])

    # palette_distances = {
    #     "Spring": average_distance(closest_spring_colors),
    #     "Summer": average_distance(closest_summer_colors),
    #     "Autumn": average_distance(closest_autumn_colors),
    #     "Winter": average_distance(closest_winter_colors)
    # }

    # closest_palette = min(palette_distances, key=palette_distances.get)

    # # Print results
    # print(f"Combined Feature Color: {combined_color}")
    # print(f"Closest Spring Colors: {closest_spring_colors}, Average Distance: {palette_distances['Spring']:.2f}")
    # print(f"Closest Summer Colors: {closest_summer_colors}, Average Distance: {palette_distances['Summer']:.2f}")
    # print(f"Closest Autumn Colors: {closest_autumn_colors}, Average Distance: {palette_distances['Autumn']:.2f}")
    # print(f"Closest Winter Colors: {closest_winter_colors}, Average Distance: {palette_distances['Winter']:.2f}")
    # print(f"Closest Overall Palette: {closest_palette}\n")

    # # Visualize the combined average color and the closest palette
    # visualize_color(combined_color, "Combined Average Color")

    # selected_palette = {
    #     "Spring": spring_palette,
#         "Summer": summer_palette,
#         "Autumn": autumn_palette,
#         "Winter": winter_palette
#     }[closest_palette]

#     visualize_palette(selected_palette, f"Closest Palette: {closest_palette}")

# # Example feature colors (you can replace these with actual feature colors)


# # Aggregate facial feature colors into a list
# feature_colors = [
#     average_skin_color,
#     average_brows_color,
#     average_eyes_color,
#     average_lips_color,
#     average_hair_color
# ]

# # Analyze the combined facial feature colors using KNN
# analyze_combined_palette_knn(feature_colors, k=3)

In [ ]:
# 1.	The code first calculates the Euclidean distance between colors to measure similarity.
# 	2.	It identifies the k closest colors to a target color from a predefined color palette using KNN.
# 	3.	It visualizes colors and palettes for better understanding.
# 	4.	It combines the average colors of various facial features and analyzes which predefined palette matches best.
# 	5.	Finally, it visualizes the results to show the closest palette and the average combined color.

# This code effectively combines color analysis techniques with visualization to understand
# the overall color theme of facial features and match them to predefined palettes,
# which could be useful in fields like fashion, cosmetics, and personal styling.

In [ ]:
visualize_color(average_hair_color, "hair")

In [ ]:
visualize_color(average_brows_color, "brows")

In [ ]:
visualize_color(average_lips_color, "lips")

In [ ]:
visualize_color(average_skin_color, "skin")

In [ ]:
visualize_color(average_eyes_color, "eyes")

In [ ]:
# Using skin as an indicator for the branches of color being warm, cool or neutral.

average_skin_color
visualize_color(average_skin_color, "skin")

In [ ]:
warm_palette_pop = [
    (196, 146, 107),  # RGB color: 196 146 107
    (205, 158, 123),  # RGB color: 205 158 123
    (214, 171, 138),  # RGB color: 214 171 138
    (222, 183, 154),  # RGB color: 222 183 154
    (231, 196, 169),  # RGB color: 231 196 169
    (240, 208, 185),  # RGB color: 240 208 185

    # Adding brighter warm reds
    (255, 120, 100),  # Bright warm red
    (255, 80, 70),    # Vivid warm red
    (255, 60, 50),    # Intense warm red

    # Additional bright warm reds
    (255, 102, 85),   # Warm salmon red
    (255, 87, 34),    # Warm orange-red
    (255, 127, 80),   # Warm tangerine
    (255, 99, 71),    # Warm tomato red
]

In [ ]:
cool_palette_pop = [
    (176, 132, 118),  # Cool beige with a pinkish tint
    (192, 146, 133),  # Slightly darker cool beige with pink undertones
    (207, 162, 143),  # Cool peachy beige
    (224, 178, 157),  # Light cool peach
    (236, 190, 169),  # Soft cool peach
    (244, 203, 182),  # Light cool peach

    # Adding cooler red and blueish tones
    (255, 180, 180),  # Light cool pink
    (255, 140, 140),  # Vivid cool pink
    (200, 100, 100),  # Cool red
    (150, 200, 255),  # Light blueish tint
    (100, 150, 255),  # Bright blueish tint

    # Additional cool tones with more purple and darker shades

    (120, 80, 160),   # Deep purple
    (100, 50, 150),   # Darker violet
    (80, 40, 120),    # Very dark purple
    (60, 30, 100),    # Rich deep purple

]

In [ ]:
def display_colors(color_palette):
    # Number of colors
    num_colors = len(color_palette)

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(num_colors, 2))  # Adjust figsize to fit all colors

    # Create an array of color blocks
    color_blocks = np.arange(num_colors)
    color_values = np.array(color_palette) / 255  # Normalize RGB values to [0, 1]

    # Display each color as a block
    ax.bar(color_blocks, height=1, width=1, color=color_values, edgecolor='none')

    # Remove x and y ticks
    ax.set_xticks([])
    ax.set_yticks([])

    # Set limits to ensure colors are displayed correctly
    ax.set_xlim(0, num_colors)
    ax.set_ylim(0, 1)

    # Add title
    ax.set_title('Color Palette')

    # Show the plot
    plt.show()

In [ ]:
display_colors(warm_palette_pop)

In [ ]:
display_colors(cool_palette_pop)

In [ ]:
closest_tone_warm = knn_closest_colors(target_color=average_skin_color, color_palette=warm_palette_pop)

closest_tone_cool = knn_closest_colors(target_color=average_skin_color, color_palette=cool_palette_pop)


In [ ]:
# Using skin as an indicator for the branches of color being warm, cool or neutral.

average_skin_color
visualize_color(average_skin_color, "skin")

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def analyze_combined_palette_knn(feature_colors, warm_palette, cool_palette, k=3, distance_threshold=10):
    """
    Analyzes and finds the K closest colors from predefined palettes to the skin using KNN.

    Args:
    feature_colors (list): List of RGB values of the facial feature colors.
    warm_palette (list): List of RGB colors for warm tones.
    cool_palette (list): List of RGB colors for cool tones.
    k (int): Number of nearest neighbors to consider.

    Returns:
    tuple: (classification, avg_warm_distance, avg_cool_distance)
    classification (str): Classification of the tone as 'warm' or 'cool'.
    avg_warm_distance (float): Average distance to the warm palette.
    avg_cool_distance (float): Average distance to the cool palette.
    """

    # Find K closest colors in the warm and cool palettes
    closest_tone_warm = knn_closest_colors(target_color=feature_colors, color_palette=warm_palette, k=k)
    closest_tone_cool = knn_closest_colors(target_color=feature_colors, color_palette=cool_palette, k=k)

    # Calculate average distances to the K closest colors
    warm_distances = [euclidean_distances([feature_colors], [color])[0][0] for color in closest_tone_warm]
    cool_distances = [euclidean_distances([feature_colors], [color])[0][0] for color in closest_tone_cool]

    avg_warm_distance = np.mean(warm_distances)
    avg_cool_distance = np.mean(cool_distances)

    # Determine the classification based on the absolute difference in average distances
    if abs(avg_warm_distance - avg_cool_distance) < distance_threshold:
        classification = 'neutral'
    elif avg_warm_distance < avg_cool_distance:
        classification = 'warm'
    else:
        classification = 'cool'

    # Output classification and distances
    print(f"Classification: {classification}")
    print(f"Average distance to warm palette: {avg_warm_distance}")
    print(f"Average distance to cool palette: {avg_cool_distance}")



    return classification, avg_warm_distance, avg_cool_distance


In [ ]:
# using redness as an indicator
tone, avg_warm_distance, avg_cool_distance = analyze_combined_palette_knn(average_skin_color, warm_palette, cool_palette, k=5)

In [ ]:
warm_palette_skin_tones = [
    (240, 225, 209),  # Light warm tone
    (235, 205, 158),  # Light-medium warm tone
    (217, 175, 123),  # Medium warm tone
    (192, 142, 93),   # Medium-dark warm tone
    (144, 100, 63),   # Dark warm tone
    (95, 60, 38),     # Deep warm tone
]

cool_palette_skin_tones = [
    (235, 220, 220),  # Light cool tone
    (217, 200, 185),  # Light-medium cool tone
    (200, 175, 160),  # Medium cool tone
    (160, 130, 115),  # Medium-dark cool tone
    (110, 80, 80),    # Dark cool tone
    (75, 55, 55),     # Deep cool tone
]

In [ ]:
display_colors(warm_palette_skin_tones)

In [ ]:
display_colors(cool_palette_skin_tones)

In [ ]:
tone, avg_warm_distance, avg_cool_distance = analyze_combined_palette_knn(average_skin_color, warm_palette=warm_palette_skin_tones, cool_palette=cool_palette_skin_tones, k=5)

Comparison between warm and colors that pop and actual skin tone.

In [ ]:
def analyze_combined_palette_knn(feature_colors, warm_skin_palette, cool_skin_palette, warm_dominant_palette, cool_dominant_palette, k=3, weight_skin=0.6, weight_dominant=0.4):
    """
    Analyzes and finds the K closest colors from predefined palettes to the skin using KNN, combining both
    actual skin tone palettes and dominant color palettes.

    Args:
    feature_colors (list): List of RGB values of the facial feature colors.
    warm_skin_palette (list): List of RGB colors for warm skin tones.
    cool_skin_palette (list): List of RGB colors for cool skin tones.
    warm_dominant_palette (list): List of RGB colors that are dominant in warm tones.
    cool_dominant_palette (list): List of RGB colors that are dominant in cool tones.
    k (int): Number of nearest neighbors to consider.
    weight_skin (float): Weight for the actual skin tone palette.
    weight_dominant (float): Weight for the dominant color palette.

    Returns:
    str: Classification of the tone as 'warm', 'cool', or 'neutral'.
    """

    # Find K closest colors in each palette
    closest_skin_warm = knn_closest_colors(feature_colors, warm_skin_palette, k=k)
    closest_skin_cool = knn_closest_colors(feature_colors, cool_skin_palette, k=k)
    closest_dom_warm = knn_closest_colors(feature_colors, warm_dominant_palette, k=k)
    closest_dom_cool = knn_closest_colors(feature_colors, cool_dominant_palette, k=k)

    # Calculate average distances to the K closest colors
    warm_skin_distances = [np.linalg.norm(np.array(feature_colors) - np.array(color)) for color in closest_skin_warm]
    cool_skin_distances = [np.linalg.norm(np.array(feature_colors) - np.array(color)) for color in closest_skin_cool]
    warm_dom_distances = [np.linalg.norm(np.array(feature_colors) - np.array(color)) for color in closest_dom_warm]
    cool_dom_distances = [np.linalg.norm(np.array(feature_colors) - np.array(color)) for color in closest_dom_cool]

    avg_warm_skin_distance = np.mean(warm_skin_distances)
    avg_cool_skin_distance = np.mean(cool_skin_distances)
    avg_warm_dom_distance = np.mean(warm_dom_distances)
    avg_cool_dom_distance = np.mean(cool_dom_distances)

    # Weighted average of distances
    avg_warm_distance = weight_skin * avg_warm_skin_distance + weight_dominant * avg_warm_dom_distance
    avg_cool_distance = weight_skin * avg_cool_skin_distance + weight_dominant * avg_cool_dom_distance

    # Determine the closest overall palette
    difference = abs(avg_warm_distance - avg_cool_distance)

    if difference < 10:  # Threshold for neutral classification
        print("The tone is classified as neutral.")
        return 'neutral'
    elif avg_warm_distance < avg_cool_distance:
        print(f"The tone is classified as warm. Avg warm distance: {avg_warm_distance}, Avg cool distance: {avg_cool_distance}")
        return 'warm'
    else:
        print(f"The tone is classified as cool. Avg warm distance: {avg_warm_distance}, Avg cool distance: {avg_cool_distance}")
        return 'cool'

In [ ]:
analyze_combined_palette_knn(
    feature_colors= average_skin_color ,
    warm_skin_palette= warm_palette_skin_tones,
    cool_skin_palette= cool_palette_skin_tones,
    warm_dominant_palette= warm_palette_pop,
    cool_dominant_palette= cool_palette_pop,
    k= 3,
    weight_skin= 0.6,
    weight_dominant= 0.4
)